In [5]:
import random
import numpy as np  

In [6]:
def cyclic_right_shift(num, k, n):
    return (num >> k) | (num << (n - k))

def cyclic_left_shift(num, k, n):
    return (num << k) | (num >> (n - k))

def bits_to_bytes(a):
    i = 0
    res = []
    for i in range(0, len(a) - 1, 8):
        x = a[i:i + 8]
        y = 0
        for j in range(len(x)):
            y += 2 ** j * x[j]
        res.append(y)   
    return res

Генератори: 

In [7]:
m = 2 ** 32
a = 2 ** 16 + 1
c = 119 

#
#Вбудований:
def integrated_gen(n):
    seq = np.random.randint(256, size=(n))
    return seq

#Лемера:
def lemer_gen(state, x_0, n):
    i = 0
    x_i = 0
    while i != n:
        x_i = (a * x_0 + c) % m
        x_0 = x_i
        i += 1
    print(x_i)
    if state == "low":
        return x_i % (2**8)
    elif "high": 
        return (x_i >> 24)

#L20       
def l20_gen(t):
    seq = np.random.randint(2, size=(20))
    for j in range(20, t):
        seq = np.append(seq, (seq[j - 3] ^ seq[j - 5] ^ seq[j - 9] ^ seq[j - 20]))
    return bits_to_bytes(seq)

#L89
def l89_gen(t):
    seq = np.random.randint(2, size=(89))
    for j in range(89, t):
        seq = np.append(seq, (seq[j - 38] ^ seq[j - 89]))
    return bits_to_bytes(seq)

#Geffe
def geffe_gen(t):
    x = np.random.randint(2, size=(10))
    y = np.random.randint(2, size=(8))
    s = np.random.randint(2, size=(9))
    z = np.zeros(t)
    for i in range(len(x), t):
        x = np.append(x, (x[i - 11] ^ x[i - 9]))
    for i in range(len(y), t):
        y = np.append(y, (y[i - 9] ^ y[i - 8] ^ y[i - 7] ^ y[i - 6]))
    for i in range(len(s), t):
        s = np.append(s, (s[i - 10] ^ s[i - 7]))
    z = s & x ^ (1 ^ s) & y
    return bits_to_bytes(z)

#Wolfram
def wolfram_gen(t):
    r = []
    r.append(random.randint(1, (2 ** 32) - 1))
    seq = []
    for i in range(t):
        seq.append(r[i] % 2)
        r.append(cyclic_left_shift(r[i], 1, 32) ^ (r[i] | (cyclic_right_shift(r[i], 1, 32))))
    return seq

#Бібліотекар
def librarian_gen(n, name_f):
    f = open(name_f, 'r', encoding='utf-8')
    text = f.read()
    f.close()
    

    


In [8]:
print(wolfram_gen(15))

[1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0]
